## MERT YUKSEKGONUL
### 2016402147

In [0]:
import numpy as np

In [0]:
from google.colab import files
files.upload()

I have trained the model on Donald Trump's speeches.

In [0]:
def softmax(x):
  # Subtract max for stabilization, preventing overflow etc.  
  x_s = x-np.max(x)
  return np.exp(x_s) / np.sum(np.exp(x_s))

In [0]:
# I will use the single layered vanilla char-rnn model, with adam optimizer.

class CharRNN:
  def __init__(self, d_hidden, d_vocab, maps, seq_len = 25, w_init=0.01, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-08):
    """
    Character predictor RNN model. Author: Mert Yuksekgonul (@mertyg)
    d_hidden, d_vocab : Hidden size and vocab size, int
    maps : A list of maps. First one is expected to be a char to index map, second one is the reverse index to char map.
    seq_len : Length of windows to be considered
    w_init : Initialization constant for weight matrices.
    learning_rate : lr for adam.
    beta1, beta2, epsilon: other parameters for adam optimization
    """
    self.d_h = d_hidden
    self.d_v = d_vocab
    np.random.seed(0)
    self.Whx = np.random.randn(d_hidden, d_vocab)*w_init
    self.Whh = np.random.randn(d_hidden, d_hidden)*w_init
    self.Wyh = np.random.randn(d_vocab, d_hidden)*w_init
    self.bh = np.zeros((d_hidden, 1))
    self.by = np.zeros((d_vocab, 1))
    self.lr = learning_rate
    self.beta1 = beta1
    self.beta2 = beta2
    self.epsilon = epsilon
    self.char_ix = maps[0]
    self.ix_char = maps[1]
    self.seq_len = 25
    self.mWyh, self.vWyh = np.zeros_like(self.Wyh), np.zeros_like(self.Wyh)
    self.mWhx, self.vWhx = np.zeros_like(self.Whx), np.zeros_like(self.Whx)
    self.mWhh, self.vWhh = np.zeros_like(self.Whh), np.zeros_like(self.Whh)
    self.mby, self.vby = np.zeros_like(self.by), np.zeros_like(self.by)
    self.mbh, self.vbh = np.zeros_like(self.bh), np.zeros_like(self.bh)
  
  def _forward_cell(self, h_prev, x):
    h_next = np.tanh(np.dot(self.Whh, h_prev) + np.dot(self.Whx, x) + self.bh)
    pred = softmax(np.dot(self.Wyh, h_next) + self.by)
    return h_next, pred
  
  def _forward_pass(self, inputs, h):
    #h = np.zeros((self.d_h, 1))
    hiddens = {}
    hiddens[-1] = h
    preds = {}
    for i in range(len(inputs)):
      x_t = inputs[i]
      h, pred = self._forward_cell(hiddens[i-1], x_t)
      hiddens[i] = h
      preds[i] = pred
    return hiddens, preds
  
  
  def _backward_cell(self, dh_next, h_prev, h, x_t, p_t, t):
    dy = np.copy(p_t)
    dy[t] -= 1
    dWyh = np.dot(dy, h.T)
    dby = dy
    dh = np.dot(self.Wyh.T, dy) + dh_next
    dtanh = (1 - h ** 2) * dh
    dbh = dtanh
    dWhx = np.dot(dtanh, x_t.T)
    dh_prev = np.dot(self.Whh.T, dtanh)
    dWhh = np.dot(dtanh, h_prev.T)
    return dh_prev, dWyh, dWhx, dWhh, dby, dbh
    
  def _backward_pass(self, hiddens, preds, inputs, targets):
    dWhx, dWhh, dWyh = np.zeros_like(self.Whx), np.zeros_like(self.Whh), np.zeros_like(self.Wyh)
    dbh, dby = np.zeros_like(self.bh), np.zeros_like(self.by)
    dh_next = np.zeros_like(hiddens[0])
    for i in reversed(range(len(targets))):
      grads = self._backward_cell(dh_next, hiddens[i-1], hiddens[i], inputs[i], preds[i], targets[i])
      dh_next = grads[0]
      dWyh += grads[1]
      dWhx += grads[2]
      dWhh += grads[3]
      dby += grads[4]
      dbh += grads[5]
    grads = [dWyh, dWhx, dWhh, dby, dbh]
    for param in grads:
      np.clip(param, -5, 5, out=param)
    return grads
    
    
  def _evaluate(self,inputs, targets, prev_h):
    hiddens, preds = self._forward_pass(inputs, prev_h)
    loss = 0
    for t in range(len(targets)):
      loss += -np.log(preds[t][targets[t], 0])
    grads = self._backward_pass(hiddens, preds, inputs, targets)
      
    return loss, hiddens[len(targets)-1], grads
  
  
  def train(self, input_txt):        
    t = 0
    window = 0
    total_loss = 0.
    batches = 1
    prev_h = np.zeros((hidden_size, 1)) 
    loss_hist = []
    
    
    while True:      
      if batches % 10000 == 0:
        print("Mean loss so far: ", total_loss/batches)
        #if len(loss_hist) > 10 and total_loss/batches > 0.999*loss_hist[-1] and loss_hist[-1] > 0.999*loss_hist[-2] and loss_hist[-2] > 0.999*loss_hist[-3]:
        #  break
        loss_hist.append(total_loss/batches)
        total_loss = 0
        batches = 0
      
      if window+self.seq_len+1 >= len(input_txt):
        window = 0
        prev_h = np.zeros((hidden_size, 1))
        
      data = [self.char_ix[ch] for ch in input_txt[window : window + self.seq_len]]
      inputs = {}
      for t in range(len(data)):
        inputs[t] = np.zeros((self.d_v, 1))
        inputs[t][data[t]] = 1
        
      targets = [self.char_ix[ch] for ch in input_txt[window + 1 : window + self.seq_len + 1]]
      
      
      loss, prev_h, grads = self._evaluate(inputs, targets, prev_h)
      window += self.seq_len
      total_loss += loss
      batches += 1
      
      for param, g, m, v in zip([self.Wyh, self.Whx, self.Whh, self.by, self.bh],
                                grads,
                                [self.mWyh, self.mWhx, self.mWhh, self.mby, self.mbh],
                                [self.vWyh, self.vWhx, self.vWhh, self.vby, self.vbh]):
        t+=1
        m_prev = m
        v_prev = v
        m += self.beta1 * m_prev + (1 - self.beta1) * g - m_prev
        v += self.beta2 * v_prev + (1 - self.beta2) * np.power(g, 2) - v_prev
        m_hat = m / (1 - np.power(self.beta1, t)) + (1 - self.beta1) * g / (1 - np.power(self.beta1, t))
        v_hat = v / (1 - np.power(self.beta2, t))
        param -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
    
    print("Training Complete!")
    
    
  def generate(self, sample, sample_size):
    sample_input = {}
    for i in range(len(sample)-1):
      data = np.zeros((self.d_v, 1))
      data[self.char_ix[sample[i]]] = 1
      sample_input[i] = data
    h_prev = np.zeros((self.d_h, 1))
    hiddens, preds = self._forward_pass(sample_input, h_prev)
    h_prev = hiddens[len(sample_input)-1]
  
    x_ix = [self.char_ix[ch] for ch in sample[-1]]
    x = np.zeros((self.d_v, 1))
    x[self.char_ix[sample[-1]]] = 1
    sampled = []
    
    for i in range(sample_size):
      h_prev, pred = self._forward_cell(h_prev, x)
      pos = np.random.choice(range(self.d_v), p=pred.ravel())
      x = np.zeros((self.d_v, 1))
      x[pos] = 1
      sampled.append(self.ix_char[pos])
    
    sample_txt = "".join(sampled)
    print(sample, sample_txt)

In [0]:
data = open('./trial/speeches.txt', 'r').read()
chars = list(set(data))
vocab_size = len(chars)
hidden_size = 200
charmap = { ch:i for i,ch in enumerate(chars) }
ixmap = { i:ch for i,ch in enumerate(chars) }
model = CharRNN(hidden_size, vocab_size, [charmap, ixmap], learning_rate=0.001, seq_len=seq_length)
history = model.train(data)

In [0]:
model.generate("America", sample_size = 1500)

America . You can’t wait paying to killeh if you can be ablanduy. I think Geved that ban. Hillary trying groral credy – I back free seem now speakise in Tom for a vamberractare issued? You know to do sore and it guy two moner. O




These are the Debate, foruse – some people who do this anywhere it come, doan, way rego. Prowes ups contrnor. I was doing out by sought this doesn’t have to do? I took for a raving him byin "I think way a young from our doll." I said they trilling down is tor your manut thing. And we have not see for mbailed to do the many, many of it. Can Brad worata. It’s sitticks that Hircaistce for a long time both solen. I say the people – he last thing to do so everythical.
Clinated it lastens. They said, "But you want to hambe the history and maybe these competently. We’re going to rume how states. We did. But she said "What a ro do, womon.
" for 41% and when do semendless wigh carrina, we swer. I said, "What have to bone.
So this told Trumparatice." I’ll the enstwar

In [0]:
model.generate("Your", sample_size = 1500)

Your  agoundster a president.
You know the veterankly, angry it. Well, I have to do the common From. We’re great pulitically 45 mangef long in him just military – you know, like some of the end our raidg.
I know the kind your was much annories powed in New Hameshing anyther to come in was Bublina, I heard about it as they do. I would beson gent, certements to gever for so taking like them. I think it’s the esticiale. But we talk about anything it with 5% an said, "Well, of the bolly was success. They don’t lore I don’t know any montinepsest in threartes, wisnor than unementth.
And that’s to making killing to be yourreds of doesn’t suck the Bunzy wrong in. And they didn’t be for a long time, to just walk.
So, you atrong it.
And thinks about many of coupped forget is very excailly any big laterults. And acroughter.
There’s some of to funcriviculfralicals – anstee trate debate. But it down. And soec Catcord out in Fox. So many. You know who’s profors that When Israel doestieve we have to 

In [0]:
model.generate("Go", sample_size = 1500)

Go us become out sole out right?
But they have no from Muss advinities.
But I wasth? You’re sord or treates for this and rape turn’t happen. I love.
So they didn’t see the same approved a yeat?" It that stries to stop said Are in Iowa?" And then we have to expelsed and just wasn’t have to use that could bean in the other thinkwork – I took who it were sort of money to or build. Will mess it would have see hupe in bad? I said "These in Scumings to I was think it’s the Bingunes on heatinest better good jobs. I love Trump. I heper and everybody.
Now, alowere buildicelably rediticals our scement.
You know, the other dived, the Deal Hillary in only been was gine and to do. And I scates. They using Aur. Plints this day" Some of the White Horved in the Alankloy.
Mark our counsing your loppanics. When I soot.






The stopa like. But I say they looking mouth was gaiting History at the cozrane.
My your dourast to the America Iobalie than stregite, and then you know Mexico. When Ickionips you r